In [1]:
import json
import pandas as pd
import nltk
from tqdm import tqdm
nltk.download('punkt')
import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
# Conexion de la API para extraccion de informacion 
from google_play_scraper import Sort, reviews_all, app,reviews

%matplotlib inline
%config InlineBackend.figure_format ='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ericm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ericm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
app_packages = [
'mx.com.procesar.aforemovil.citi',
'mx.com.procesar.aforemovil.nuevosura',
'com.Sitio',
'mx.com.afore.xxi.banorte',
'mx.com.procesar.aforemovil.invercap',
'mx.com.procesar.aforemovil',
'com.americanexpress.android.acctsvcs.us',
'net.veritran.afmx.mb.s.prod',
'com.mifel.mobile.activity',
'mx.actinver.actinver',
'com.actinver.eactinver',
'com.autofin.bancamovil',
'mx.com.bancoazteca.bazdigitalmovil',
'mx.com.bancaempresarialazteca.appbea1',
'com.bancrea.bancreamovil',
'net.veritran.bbmx.mb.s.prod',
'com.compartamos.mobileApp',
'org.microemu.android.model.common.VTUserApplicationBNBJMB',
'mx.com.bb.b2',
'conecta.co.finterra',
'com.inbursa.client',
'mx.com.invex.tarjetas.movil',
'com.monex.android',
'com.multiva.saldosv2',
'org.microemu.android.model.common.VTUserApplicationMLTVMB',
'com.multiva.bmdc',
'com.cloudsourceit.banregio',
'com.bancsabadell.wallet',
'mx.bancsabadell.part',
'mx.bancosantander.supermovil',
'mx.com.santander.superwallet',
'com.planetmedia.bancamovil.bx2',
'mx.com.miapp',
'com.bankaool.movil',
'mx.com.glider.spiral.mobile.spiral_app.mlb',
'org.microemu.android.model.common.VTUserApplicationBNRTMB',
'com.bansi.bel.movil',
'com.bancomer.mbanking',
'com.bbva.GEMA',
'com.cibanco.android',
'com.cibanco.cicash360',
'com.citibanamex.banamexmobile',
'com.citibanamex.bnemobile',
'mx.com.mobile.consubanco',
'com.ix.agency.consubanco_codi_android',
'com.consubanco.cbmainsuite',
'mx.com.hsbc.controltotal',
'mx.hsbc.hsbcmexico',
'com.intercamGF.IntercamBM',
'com.scotiabankmx.scotiamovil',
'mx.com.anzen.scotiaTarjetas',
'ar.com.bancar.uala',
'mx.intelifin.android.albo',
'com.marcanditasl.bnext.mex',
'com.cacaopaycard.app',
'com.cuenca.cuenca_mobile',
'com.fintonic',
'com.fondeadora.bank',
'com.Kapital.KapitalApp',
'com.kubo.app.wallet.android',
'com.nu.production',
'com.pagatodo.yaganaste',
'ai.powerup.stori',
'com.broxel.four',
'com.oysterFinancial',
'mx.klar.app',
'com.bitso.wallet',
'com.afluenta.lenders',
'com.actinver.dinn',
'com.inbursa.icasabolsa',
'mx.com.invex.fondos',
'com.techrules.portfoliotools.invex',
'com.vepormas.casadebolsa',
'com.bancomer.btrader',
'com.bancomer.bbva.bbvasave',
'com.kuspit.app',
'com.oldmutual.mobileappcorp',
'com.mex.gbmmobile.containerhomebroker',
'mx.frd.donde.movil',
'com.lendbrink.mx',
'com.promotores.bpb',
'com.ionic.crediapp',
'com.crediplatapp.android',
'com.ionicframework.credis709525',
'com.resser.movil.finbe',
'app.impulsarte.impulsarteclientes',
'mx.com.intermercadoclientes',
'com.konfio.credito',
'xyz.lana.b2c',
'mx.avasis.microfin_lanahora',
'com.resuelvetudeuda.app',
'mx.com.tala',
'com.uniclick.portalclientes',
'mx.minu.app',
'com.baubap',
'com.loan.cash.credit.nanopaymx',
'com.monific.app',
'br.com.creditas.mobile',
'es.appcreditea',
'banorte.com.peper',
'com.uno.hdi.contigo',
'com.xs.protect.woow',
'com.metlife.mexico.business.contigo',
'com.banorte.seguros',
'com.Cardif.ScotiabankSalud',
'com.allianzworldwidecare.mobile',
'com.allianz.allianzapp',
'com.blacksquared.changers.allianz',
'com.seguros.anago',
'www.segurosatlas.ionic',
'mx.axa.mobile.customer',
'mx.com.santander.SantanderSeguros',
'mx.com.segurosbancomer.autoalerta',
'com.vitalityactive.mexicoVitality',
'com.bancomer.bbva.wibe',
'org.mexico.bupa',
'com.abachubb.appsiniestros',
'com.crabi.customerjourneys',
'com.dentegra.seguros',
'com.scope.seguroxuso',
'com.cra.generaldeseguros',
'com.gnp',
'com.mapfre.selfmex',
'com.MAC.afiliados.mobil',
'com.MI_PLAN_SEGURO',
'com.principal.fondosinversion',
'mx.com.qualitas.QMovil',
'mx.com.segurosazteca',
'mx.habil.se.seguros.centauro.clientes.android.app',
'com.elpotosi.android',
'com.medic.medicapp',
'com.zurich.ziomob2',
'com.bentel.sisnova',
'com.latino.app.lalatinoseguros2',
'com.insigniaseguros',
'mx.zurich.mobile',
'com.novo.cpm',
'com.mercadopago.wallet',
'com.saldo.nope.main',
'com.sofiasalud.app',
'com.suraam.qiip',
'com.vepormas.wallet',
'com.banorte.codicomercios.dependiente',
'com.bansi.pmovil.movil',
'com.bansi.codimovil.android',
'com.bbvasend.inicio',
'com.payclip.clip',
'edenred.mx.mobile.services',
'mx.finerio',
'com.enso.app',
'mx.leasemd.app',
'com.mexpago.wallet.app',
'com.previvale.com.mx',
'agency.equilibrio.sodexomexico',
'mx.toka.app',
'mx.com.bancoazteca.wallet.cliente',
'com.conekta.checkout',
'com.incomepos.registerandroid',
'com.moneypoolmobile',
'com.cobra.srpagopos',
'mx.com.netpay.app',
'com.walmartmexico.wallet',
'com.pagopopmobile',
'com.globalhitss.claro.pay',
'com.kdm.apps.miio',
'mx.blacklabs.pagandowallet',
'com.autochilango',
'com.asistensiMX.app',
'com.rym_solutions.avant',
'io.bdeo.app',
'com.grupobituaj.clickseguros',
'mx.com.clupp.autoclupp',
'com.jooycar.paypermile',
'com.jooycar.Fleetr',
'com.jooycar.consorcioB2C',
'com.jooycar.paypermile.consorcio',
'mx.meddi.app',
'com.meetingdoctors.chatmedico',
'com.heruapp',
'com.banregio.hey'
]

In [3]:
app_infos = []

for ap in tqdm(app_packages):
    print(ap)
    try:
        info = app(ap, lang='es', country='mx')
        del info['comments']
        app_infos.append(info)
    except:
        print(ap+' - No se encontro. Siguiente.');

  0%|          | 0/180 [00:00<?, ?it/s]

mx.com.procesar.aforemovil.citi


  1%|          | 1/180 [00:00<00:49,  3.58it/s]

mx.com.procesar.aforemovil.nuevosura


  1%|          | 2/180 [00:00<00:48,  3.69it/s]

...

 99%|█████████▉| 178/180 [00:57<00:00,  3.22it/s]

com.heruapp


 99%|█████████▉| 179/180 [00:58<00:00,  3.19it/s]

com.banregio.hey


100%|██████████| 180/180 [00:58<00:00,  3.08it/s]


In [4]:
def print_json(json_object):
    json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str)
    
    print(highlight(json_str, JsonLexer(), TerminalFormatter()))
    print_json(app_infos[0])

In [5]:
app_infos_df = pd.DataFrame(app_infos)

In [6]:
app_reviews_original = []

for ap in tqdm(app_packages):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs,_ = reviews(
            ap,
            lang='es',
            country='mx',
            count=12000,
            sort=sort_order
            )
    for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
    app_reviews_original.extend(rvs)

100%|██████████| 180/180 [35:21<00:00, 11.79s/it] 


In [8]:
app_reviews = app_reviews_original
stop_words_custom = [',','!','?','¡','¿','.','a','actualmente','adelante','puro', 'además','afirmó','agregó','ahora','ahí','al','algo','alguna','algunas','alguno','algunos','algún','alrededor','ambos','ampleamos','ante','anterior','antes','apenas','aproximadamente','aquel','aquellas','aquellos','aqui','aquí','arriba','aseguró','así','atras','aunque','ayer','añadió','aún','bajo','bastante','buen','bueno','buenos','cada','casi','cerca','cierta','ciertas','cierto','ciertos','cinco','comentó','como','con','conocer','conseguimos','conseguir','considera','consideró','consigo','consigue','consiguen','consigues','contra','cosas','creo','cual','cuales','cualquier','cuando','cuanto','cuatro','cuenta','cómo','da','dado','dan','dar','de','debe','deben','debido','decir','dejó','del','demás','dentro','desde','después','dice','dicen','dicho','dieron','diferente','diferentes','dijeron','dijo','dio','donde','dos','durante','e','ejemplo','el','ella','ellas','ello','ellos','embargo','empleais','emplean','emplear','empleas','empleo','en','encima','encuentra','entonces','entre','era','erais','eramos','eran','eras','eres','es','esa','esas','ese','eso','esos','esta','estaba','estabais','estaban','estabas','estad','estada','estadas','estado','estados','estais','estamos','estan','estando','estar','estaremos','estará','estarán','estarás','estaré','estaréis','estaría','estaríais','estaríamos','estarían','estarías','estas','este','estemos','esto','estos','estoy','estuve','estuviera','estuvierais','estuvieran','estuvieras','estuvieron','estuviese','estuvieseis','estuviesen','estuvieses','estuvimos','estuviste','estuvisteis','estuviéramos','estuviésemos','estuvo','está','estábamos','estáis','están','estás','esté','estéis','estén','estés','ex','existe','existen','explicó','expresó','fin','fue','fuera','fuerais','fueran','fueras','fueron','fuese','fueseis','fuesen','fueses','fui','fuimos','fuiste','fuisteis','fuéramos','fuésemos','gran','grandes','gueno','ha','haber','habida','habidas','habido','habidos','habiendo','habremos','habrá','habrán','habrás','habré','habréis','habría','habríais','habríamos','habrían','habrías','habéis','había','habíais','habíamos','habían','habías','hace','haceis','hacemos','hacen','hacer','hacerlo','haces','hacia','haciendo','hago','han','has','hasta','hay','haya','hayamos','hayan','hayas','hayáis','he','hecho','hemos','hicieron','hizo','hoy','hube','hubiera','hubierais','hubieran','hubieras','hubieron','hubiese','hubieseis','hubiesen','hubieses','hubimos','hubiste','hubisteis','hubiéramos','hubiésemos','hubo','igual','incluso','indicó','informó','intenta','intentais','intentamos','intentan','intentar','intentas','intento','ir','junto','la','lado','largo','las','le','les','llegó','lleva','llevar','lo','los','luego','lugar','manera','manifestó','mayor','me','mediante','mejor','mencionó','menos','mi','mientras','mio','mis','misma','mismas','mismo','mismos','modo','momento','mucha','muchas','mucho','muchos','muy','más','mí','mía','mías','mío','míos','nada','nadie','ni','ninguna','ningunas','ninguno','ningunos','ningún','no','nos','nosotras','nosotros','nuestra','nuestras','nuestro','nuestros','nueva','nuevas','nuevo','nuevos','nunca','o','ocho','os','otra','otras','otro','otros','para','parece','parte','partir','pasada','pasado','pero','pesar','poca','pocas','poco','pocos','podeis','podemos','poder','podria','podriais','podriamos','podrian','podrias','podrá','podrán','podría','podrían','poner','por','por qué','porque','posible','primer','primera','primero','primeros','principalmente','propia','propias','propio','propios','próximo','próximos','pudo','pueda','puede','pueden','puedo','pues','que','quedó','queremos','quien','quienes','quiere','quién','qué','realizado','realizar','realizó','respecto','sabe','sabeis','sabemos','saben','saber','sabes','se','sea','seamos','sean','seas','segunda','segundo','según','seis','ser','seremos','será','serán','serás','seré','seréis','sería','seríais','seríamos','serían','serías','seáis','señaló','si','sido','siempre','siendo','siete','sigue','siguiente','sin','sino','sobre','sois','sola','solamente','solas','solo','solos','somos','son','soy','su','sus','suya','suyas','suyo','suyos','sí','sólo','tal','también','tampoco','tan','tanto','te','tendremos','tendrá','tendrán','tendrás','tendré','tendréis','tendría','tendríais','tendríamos','tendrían','tendrías','tened','teneis','tenemos','tener','tenga','tengamos','tengan','tengas','tengo','tengáis','tenida','tenidas','tenido','tenidos','teniendo','tenéis','tenía','teníais','teníamos','tenían','tenías','tercera','ti','tiempo','tiene','tienen','tienes','toda','todas','todavía','todo','todos','total','trabaja','trabajais','trabajamos','trabajan','trabajar','trabajas','trabajo','tras','trata','través','tres','tu','tus','tuve','tuviera','tuvierais','tuvieran','tuvieras','tuvieron','tuviese','tuvieseis','tuviesen','tuvieses','tuvimos','tuviste','tuvisteis','tuviéramos','tuviésemos','tuvo','tuya','tuyas','tuyo','tuyos','tú','ultimo','un','una','unas','uno','unos','usa','usais','usamos','usan','usar','usas','uso','usted','va','vais','valor','vamos','van','varias','varios','vaya','veces','ver','verdad','verdadera','verdadero','vez','vosotras','vosotros','voy','vuestra','vuestras','vuestro','vuestros','y','ya','yo','él','éramos','ésta','éstas','éste','éstos','última','últimas','último','últimos','app','veces','aplicación','use','m','n','o','1','2','3','4','5','6','7','8','9','0','10','100','20','não','a','si','la','lo','Ya','y','vez','don','se','va','12','11','1st','na','da','em','que','go','por','de','é','lot','up','um','dos','cada','com','ver','muy','es','el','de','a','o','que','e','do','da','em','um','para','é','com','não','uma','os','no','se','na','por','mais','as','dos','como','mas','foi','ao','ele','das','tem','à','seu','sua','ou','ser','quando','muito','há','nos','já','está','eu','também','só','pelo','pela','até','isso','ela','entre','era','depois','sem','mesmo','aos','ter','seus','quem','nas','me','esse','eles','estão','você','tinha','foram','essa','num','nem','suas','meu','às','minha','têm','numa','pelos','elas','havia','seja','qual','será','nós','tenho','lhe','deles','essas','esses','pelas','este','fosse','dele','tu','te','vocês','vos','lhes','meus','minhas','teu','tua','teus','tuas','nosso','nossa','nossos','nossas','dela','delas','esta','estes','estas','aquele','aquela','aqueles','aquelas','isto','aquilo','estou','está','estamos','estão','estive','esteve','estivemos','estiveram','estava','estávamos','estavam','estivera','estivéramos','esteja','estejamos','estejam','estivesse','estivéssemos','estivessem','estiver','estivermos','estiverem','hei','há','havemos','hão','houve','houvemos','houveram','houvera','houvéramos','haja','hajamos','hajam','houvesse','houvéssemos','houvessem','houver','houvermos','houverem','houverei','houverá','houveremos','houverão','houveria','houveríamos','houveriam','sou','somos','são','era','éramos','eram','fui','foi','fomos','foram','fora','fôramos','seja','sejamos','sejam','fosse','fôssemos','fossem','for','formos','forem','serei','será','seremos','serão','seria','seríamos','seriam','tenho','tem','temos','tém','tinha','tínhamos','tinham','tive','teve','tivemos','tiveram','tivera','tivéramos','tenha','tenhamos','tenham','tivesse','tivéssemos','tivessem','tiver','tivermos','tiverem','terei','terá','teremos','terão','teria','teríamos','teriam']

In [9]:
for ar in app_reviews:
  try:
    ar["at_timestamp"] = ar["at"].strftime("%m/%d/%Y %H:%M:%S")
  except:
    ar["at"] = " "

In [10]:
for ar in app_reviews:
  try:
    ar["contentFiltered"] = word_tokenize(ar["content"].lower())
    ar["contentFiltered"] = [w for w in ar["contentFiltered"] if not w in stop_words_custom]
  except:
    ar["contentFiltered"] = " "

def print_json2(json_object):
    json_str2 = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
print(highlight(json_str2, JsonLexer(), TerminalFormatter())
print_json2(app_reviews[0])

SyntaxError: invalid syntax (Temp/ipykernel_19748/3809965209.py, line 16)

In [11]:
for ar in app_reviews_original:
    try:
        ar["wordCloud"]= ' '.join(ar["contentFiltered"])
    except:
        ar["wordCloud"] = " "
print(app_reviews[0])

{'reviewId': 'gp:AOqpTOEp9g57ig9zuHV2krMsSbckZwPVyTYs99amm_-mlK96xDPoqsqH6Mzb5B-H5WVbIY_hKu5MM6BpVYo6TQ', 'userName': 'rulasblack', 'userImage': 'https://play-lh.googleusercontent.com/a-/AOh14Gg_Q_hndKoUsROidRJd9XclXP-8wOqMwvfMDW-s', 'content': 'Está bien ala app Ya que lleva el registro Pero estaría bien si fueran opción de retiro Aceves se ocupa un poco de ayuda Que lenpaso con la actualización', 'score': 2, 'thumbsUpCount': 0, 'reviewCreatedVersion': '5.0.0', 'at': datetime.datetime(2022, 3, 31, 9, 24, 55), 'replyContent': None, 'repliedAt': None, 'sortOrder': 'newest', 'appId': 'mx.com.procesar.aforemovil.citi', 'at_timestamp': '03/31/2022 09:24:55', 'wordCloud': ' '}


In [13]:
len(app_reviews_original)

493516

In [14]:
app_reviews_df = pd.DataFrame(app_reviews_original)

In [15]:
app_reviews_df.to_csv('android_reviews_12k_2022.csv', index=None, header=True,encoding='utf-8-sig')